In [ ]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, sunday_to_monday, MO
from pandas.tseries.offsets import Day, CustomBusinessDay, DateOffset
import re

### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [ ]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    def __init__(self):
        cal = pd.read_csv('https://data.gov.ru/opendata/7708660670-proizvcalendar/data-20191112T1252-structure-20191112T1247.csv?encoding=UTF-8')
        #cal_year = my_year
        #cal = cal[cal['Год/Месяц']==cal_year]
        weekend = []
        day_to_holiday = []
        for i in range(1, 13):
            for j in re.split(r'[,+|\++]', str(cal[cal.columns[i]]).split()[1]):
                if ('*' not in j) and (len(j)>0):
                    weekend.append((i, int(j)))
                elif ('*' in j):
                    day_to_holiday.append((i, int(j[:(len(j)-1)])))
        self.weekend, self.day_to_holiday = weekend, day_to_holiday
        self.holidays = [(1,1), (1,2), (1,3), (1,4), (1,5), (2,23), (3,8),
                         (5,1), (5,9), (6,12), (11,4)]
        
    cal = pd.read_csv('https://data.gov.ru/opendata/7708660670-proizvcalendar/data-20191112T1252-structure-20191112T1247.csv?encoding=UTF-8')
    start_date = datetime.datetime(cal['Год/Месяц'][0], 1, 1)
    
    end_date = datetime.datetime(cal['Год/Месяц'][len(cal['Год/Месяц'])-1], 12, 31)
    weekmask = 'Mon Tue Wed Thu Fri'

In [ ]:
events = pd.date_range(start='2020-01-01', end='2020-12-31', freq='D')
events = pd.DataFrame(events, index=events)
events[['выходной', 'праздник', 'предпраздничный рабочий день', 
        "последний день месяца", "последний рабочий день месяца", "пн", "вт", 
        "ср", "чт", "пт", "сб", "вс", "январь", "февраль", "март", "апрель", 
        "май", "июнь", "июль", "август", "сентябрь", "октябрь", "ноябрь", 
        "декабрь", "предновогодний выходной день"]] = 0
events = events.drop(columns=0)

In [ ]:
My_cal = RussianBusinessCalendar()

for i in events.index:
    #events['выходной'][i] = int(str(i.day) in re.split(r',+', str(df[df.columns[i.month]]).split()[1]))
    events['выходной'][i] = int((i.month, i.day) in My_cal.weekend)
    #events['праздник'][i] = int((i.month, i.day) in [(1,1), (1,2), (1,3), (1,4), (1,5), (2,23), (3,8), (5,1), (5,9), (6,12), (11,4)])
    events['праздник'][i] = int((i.month, i.day) in My_cal.holidays)
    #for x in re.split(r'[],+\+]', str(df[df.columns[i.month]]).split()[1]):
    #    if len(x)>2:
    #        events['предпраздничный рабочий день'][i] = int(i.day == int(x[:2]))
    events['предпраздничный рабочий день'][i] = int((i.month, i.day) in My_cal.day_to_holiday)
    events["последний день месяца"][i] = int(i.is_month_end)
    events['последний рабочий день месяца'][i] = int(i in (events.index + pd.offsets.BMonthEnd()))
    events.loc[i, "пн":"вс"][i.dayofweek] = 1
    events.loc[i, "январь":"декабрь"][i.month-1] = 1
    events['предновогодний выходной день'][i] = int((i.month, i.day) == My_cal.weekend[-1])
    
events.head()

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,предновогодний выходной день
2020-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-05,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# пример требуемого DataFrame 
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,предновогодний выходной день
2020-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2020-01-05,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
2020-12-28,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2020-12-29,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2020-12-30,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [ ]:
#from google.colab import files
#f = files.upload()

In [ ]:
#ts = pd.read_csv('sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], 
                 #index_col='TRADEDATE', parse_dates=True)

In [ ]:
ts = pd.read_csv('https://raw.githubusercontent.com/aleksei-riabykh/sberbank_data_analysis/master/data/sber_stocks.csv',
                 usecols=['CLOSE', 'TRADEDATE'], index_col='TRADEDATE', parse_dates=True)

In [ ]:
ts_lag.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=5).mean().shift(1))

TRADEDATE
2013-03-25        NaN
2013-03-26        NaN
2013-03-27        NaN
2013-03-28        NaN
2013-03-29        NaN
               ...   
2020-01-06    243.162
2020-01-07    242.718
2020-01-08    244.180
2020-01-09    243.950
2020-01-10    245.624
Freq: D, Name: CLOSE, Length: 2483, dtype: float64

In [ ]:
my_ts = ts
my_ts= my_ts.asfreq('1D').fillna(method = 'ffill')
my_ts['weekday'] = my_ts.index.dayofweek
my_ts['Day_type'] = 0
for i in my_ts.index:
    my_ts['Day_type'][i] = str('Holiday'*int((i.month, i.day) in My_cal.holidays)+'Work'*int((i.month, i.day) not in My_cal.weekend)
                                +'_before hol'*int((i.month, i.day) in My_cal.day_to_holiday)
                               +'Weekend'*int((i.month, i.day) in My_cal.weekend)*((i.month, i.day) not in My_cal.holidays))
ts_lag = my_ts
for i in range(1,6,2):
    my_ts = my_ts.merge(ts_lag.shift(i, freq = pd.offsets.BDay(1)).drop(columns=['weekday', 'Day_type']), how = 'left', left_on = 'TRADEDATE', right_on = 'TRADEDATE')

#my_ts.columns = ['CLOSE', 'weekday', 'Day_type', 'lag_1', 'lag_3', 'lag_5']

for j in range(1,6,2):
    my_ts['roll_'+str(j)+'_mean'] = ts_lag.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=j).mean().shift(1))
    my_ts['roll_' + str(j) + '_median'] = ts_lag.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=j).median().shift(1))
    my_ts['roll_' + str(j) + '_max'] = ts_lag.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=j).max().shift(1))
    my_ts['roll_' + str(j) + '_min'] = ts_lag.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=j).min().shift(1))
    my_ts['roll_' + str(j) + '_std'] = ts_lag.groupby('weekday')['CLOSE'].transform(lambda x: x.rolling(window=j).std().shift(1))

#for j in range(1,6,2):
my_ts['expand_mean'] = ts_lag.groupby('Day_type')['CLOSE'].transform(lambda x: x.expanding().mean().shift(1))
my_ts['expand_median'] = ts_lag.groupby('Day_type')['CLOSE'].transform(lambda x: x.expanding().median().shift(1))
my_ts['expand_max'] = ts_lag.groupby('Day_type')['CLOSE'].transform(lambda x: x.expanding().max().shift(1))
my_ts['expand_min'] = ts_lag.groupby('Day_type')['CLOSE'].transform(lambda x: x.expanding().min().shift(1))
my_ts['expand_std'] = ts_lag.groupby('Day_type')['CLOSE'].transform(lambda x: x.expanding().std().shift(1))

my_ts = my_ts.fillna(method = 'bfill')
my_ts = my_ts.fillna(method = 'ffill')

my_ts = my_ts.drop_duplicates()
my_ts.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,CLOSE_x,weekday,Day_type,CLOSE_y,CLOSE_x,CLOSE_y,roll_1_mean,roll_1_median,roll_1_max,roll_1_min,roll_1_std,roll_3_mean,roll_3_median,roll_3_max,roll_3_min,roll_3_std,roll_5_mean,roll_5_median,roll_5_max,roll_5_min,roll_5_std,expand_mean,expand_median,expand_max,expand_min,expand_std
TRADEDATE,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-03-25,98.79,0,Work,98.79,98.79,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,98.790000,98.790,98.79,98.79,1.124300
2013-03-26,97.20,1,Work,98.79,98.79,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,98.790000,98.790,98.79,98.79,1.124300
2013-03-27,96.75,2,Weekend,97.20,98.79,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,96.750000,96.750,96.75,96.75,1.124300
2013-03-28,98.59,3,Weekend,96.75,98.79,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,96.750000,96.750,96.75,96.75,1.124300
2013-03-29,98.76,4,Work,98.59,97.20,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,97.995000,97.995,98.79,97.20,1.124300
2013-03-30,98.76,5,Work,98.76,96.75,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,98.250000,98.760,98.79,97.20,0.909450
2013-03-31,98.76,6,Work,98.76,96.75,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,98.377500,98.760,98.79,97.20,0.785127
2013-04-01,98.07,0,Work,98.76,96.75,98.79,98.79,98.79,98.79,98.79,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,98.454000,98.760,98.79,97.20,0.701128
2013-04-02,98.93,1,Work,98.07,98.59,97.20,97.20,97.20,97.20,97.20,NaN,99.096667,98.79,100.43,98.07,1.209518,97.574,98.07,100.43,95.1,2.257594,98.390000,98.760,98.79,97.20,0.646405
